## Testing the fast class of polynomials for correctness

We verify that our class Poly works correctly by mimicking the computation in SageMath.

In [2]:
load("poly_class.sage")
import random
import time

class TestPoly():
    def __init__(self):
        self.test_setup()
    
    def test_setup(self):
        self.N = 2 ** 4
        self.Q = 2 ** 6
        self.R = PolynomialRing(ZZ, 'x')
        self.x = self.R.gen()
        self.quo = self.x ** self.N + 1
        self.R = PolynomialRing(ZZ, 'x').quotient(self.quo)
        self.RQ = PolynomialRing(Zmod(self.Q), 'x').quotient(self.quo)
        Poly.setup(self.N, self.Q)
        
    def run_test_add_mod(self):
        a = self.RQ.random_element()
        b = self.RQ.random_element()
        a_poly = Poly(a.list(), self.Q)
        assert len(a_poly) == self.N, "Polynomial length is not N"
        b_poly = Poly(b.list(), self.Q)
        sum = a + b
        sum_poly = a_poly + b_poly
        assert sum == self.RQ(sum_poly.list())
        
    def run_test_sub(self):
        a = self.R.random_element()
        b = self.R.random_element()
        a_poly = Poly(a.list(), 0)
        b_poly = Poly(b.list(), 0)
        diff = a - b
        diff_poly = a_poly - b_poly
        assert diff == self.R(diff_poly.list())
        
    def run_test_mul_mod(self):
        a = self.RQ.random_element()
        b = self.RQ.random_element()
        a_poly = Poly(a.list(), self.Q)
        b_poly = Poly(b.list(), self.Q)
        prod = a * b
        prod_poly = a_poly * b_poly
        assert prod == self.RQ(prod_poly.list())
        
    def run_test_reduce(self):
        a = self.RQ.random_element()
        a_poly = Poly(a.list(), 0)
        a_poly = a_poly % (self.Q//2)
        a = [ZZ(i) % (self.Q//2) for i in a.list()]
        assert self.R(a) == self.R(a_poly.list())
        
    def run_test_mod_switch(self):
        a = self.RQ.random_element()
        b = self.RQ.random_element()
        a_poly = Poly(a.list(), self.Q) % (self.Q//2)
        b_poly = Poly(b.list(), self.Q) % (self.Q//2)
        c_poly = (a_poly * b_poly) % self.Q
        a = [Zmod(self.Q//2)(i) for i in a.list()]
        b = [Zmod(self.Q//2)(i) for i in b.list()]
        ring = PolynomialRing(Zmod(self.Q//2), 'x').quotient(self.quo)
        a = ring(a)
        b = ring(b)
        c = [ZZ(i) % self.Q for i in (a * b)]
        assert self.R(c) == self.R(c_poly.list())
        
    def run_test_norm(self):
        assert self.Q >= 5, "Q must be at least 5"
        a = self.RQ([-2,-1,0,1])
        assert Poly(a.list(), self.Q).norm() == 2
    
    def run_test_auto(self):
        a = self.RQ.random_element()
        a_poly = Poly(a.list(), self.Q)
        def rot(index):
            if index == -1:
                rotation = self.x ** (self.N * 2 - 1)
            else:
                rotation = self.x ** (Zmod(self.N * 2)(5) ** index)
            return rotation
        assert (a.lift()(rot(-1)) % self.quo).list() == a_poly.auto_inverse().list()
        assert (a.lift()(rot(1)) % self.quo).list() == a_poly.auto5().list()
        assert (a.lift()(rot(3)) % self.quo).list() == a_poly.auto(3).list()
        
    def run_test_div(self):
        a = self.RQ.random_element()
        factor = divisors(self.Q)[:-1]
        factor = random.choice(factor)
        a_poly = Poly(a.list(), self.Q)
        a = [ZZ(i) // factor for i in a.list()]
        a_poly = a_poly / factor
        assert self.RQ(a).list() == self.RQ(a_poly.list()).list()
        
    def run_test_center_div(self):
        a = self.RQ.random_element().list()
        a_poly = Poly(a, self.Q)
        factor = random.randint(1, self.Q//2)
        for i in range(len(a)): # centering
            if a[i] >= self.Q // 2:
                a[i] = ZZ(a[i]) - self.Q
        a = [ZZ(j) // factor for j in a]
        a_poly = a_poly.center_div(factor)
        assert self.RQ(a).list() == self.RQ(a_poly.list()).list()

    def run_test_check_moduli(self):
        a = self.RQ.random_element()
        a_poly = Poly(a.list(), self.Q)
        assert a_poly.check_modulus()
        a_poly = a_poly % 0
        assert a_poly.check_modulus()
        a_poly = a_poly % (self.Q//2)
        assert a_poly.check_modulus()
        
    def run_test_scale(self):
        a = self.RQ.random_element().list()
        a_poly = Poly(a, self.Q)
        factor = divisors(self.Q)[:-1]
        factor = random.choice(factor)
        a = [round(ZZ(i) / factor) for i in a]
        a_poly = a_poly.scale(factor)
        newring = PolynomialRing(Zmod(self.Q // factor), 'x').quotient(self.quo)
        assert newring(a).list() == newring(a_poly.list()).list()
        
    def run_test_center(self):
        a = self.RQ.random_element().list()
        a_poly = Poly(a, self.Q)
        for i in range(len(a)):
            if a[i] >= self.Q // 2:
                a[i] -= self.Q
        assert a == a_poly.center().list()
        
        

IndentationError: unexpected indent (<string>, line 35)

We will now check for correctness multiple times.

In [ ]:
Test = TestPoly()
iterations = 100
for _ in range(iterations):
    Test.run_test_add_mod()
    Test.run_test_sub()
    Test.run_test_mul_mod()
    Test.run_test_reduce()
    Test.run_test_mod_switch()
    Test.run_test_norm()
    Test.run_test_auto()
    Test.run_test_div()
    Test.run_test_center_div()
    Test.run_test_check_moduli()
    Test.run_test_scale()
    Test.run_test_center()
print("All tests passed")

All tests passed


In [ ]:
import cProfile
load("poly_class.sage")

class Runtimes:
    @classmethod
    def parameters(cls, N=2**15, Q=2**1000, iter=50):
        cls.N = N
        cls.Q = Q
        assert iter > 0
        cls.i = iter
        Poly.setup(cls.N, cls.Q)
        
    def __init__(self):
        self.parameters()
        
    def random(self):
        self.a = Poly.random(self.Q)
        self.b = Poly.random(self.Q)
        
    def run(self):
        R = RealField(20)
        ms = 1000 / self.i
        div = divisors(self.Q)[1:-1]
        print(f"Running tests with N={self.N}, log(Q)={log(self.Q,2)}, and {self.i} iterations")
        print(f"Preparing...")
        self.random()
        t = time.time()
        self.array = [Poly.random2(self.Q) for _ in range(self.i)]
        print(f"Time for random polynomials: {R(ms * (time.time() - t))} ms ")
        
        ## ARITHMETIC
    
        t = time.time()
        _ = [self.a + self.array[_] for _ in range(self.i)]
        print(f"Time for addition: {R(ms * (time.time() - t))} ms ")
        print(self.a)
        
        t = time.time()
        _ = [self.a * self.array[_] for _ in range(self.i)]
        print(f"Time for multiplication: {R(ms * (time.time() - t))} ms ")
        
        t = time.time()
        _ = [self.array[_] / (random.choice(div)) for _ in range(self.i)]
        print(f"Time for scalar division: {R(ms * (time.time() - t))} ms ")
        
        t = time.time()
        _ = [self.array[_].center_div(random.randint(1, self.Q//2)) for _ in range(self.i)]
        print(f"Time for center_div: {R(ms * (time.time() - t))} ms ")
        
        t = time.time()
        _ = [self.array[_] % (random.choice(div)) for _ in range(self.i)]
        print(f"Time for modular reduction: {R(ms * (time.time() - t))} ms ")
        
        t = time.time()
        _ = [self.array[_].center() for _ in range(self.i)]
        print(f"Time for centering: {R(ms * (time.time() - t))} ms ")

        t = time.time()
        _ = [self.array[_].scale(random.choice(div)) for _ in range(self.i)]
        print(f"Time for scaling: {R(ms * (time.time() - t))} ms ")
        
        ## AUTOMORPHISMS
        
        t = time.time()
        _ = [self.array[_].auto5() for _ in range(self.i)]
        print(f"Time for auto5: {R(ms * (time.time() - t))} ms ")
        
        t = time.time()
        _ = [self.array[_].auto_inverse() for _ in range(self.i)]
        print(f"Time for auto_inverse: {R(ms * (time.time() - t))} ms ")
        
        t = time.time()
        _ = [self.array[_].auto(_) for _ in range(self.i)]
        print(f"Time for auto: {R(ms * (time.time() - t))} ms ")
        
        ## NORMS
        
        t = time.time()
        _ = [self.array[_].centered_list() for _ in range(self.i)]
        print(f"Time for centered_list: {R(ms * (time.time() - t))} ms ")
        
        t = time.time()
        _ = [self.array[_].norm() for _ in range(self.i)]
        print(f"Time for norm: {R(ms * (time.time() - t))} ms ")
        
# profiler = cProfile.Profile()
# profiler.enable()

Run = Runtimes()
Run.run()
print("All timings done")

# profiler.disable()
# profiler.print_stats(sort='time')
    

IndentationError: unexpected indent (<string>, line 35)